# Analytic Scores and Models Overview

- **Behavioural Volatility Score**: Quantifies the variability in transaction amounts and timing for each wallet. High scores indicate erratic or inconsistent activity, while low scores suggest regular, stable behavior. Derived from statistical analysis of transaction histories.

- **Revenue Proxy Score**: Estimates the potential revenue contribution of each wallet based on transaction volume, frequency, and other relevant features. This score is used to prioritize high-value users and is calculated using aggregated transaction data.

- **Cross Domain Engagement Score**: Measures how actively a wallet participates across different product or service domains (e.g., trading, staking, lending). Higher scores reflect broader engagement, derived from counting unique domain interactions in the raw data.

- **Interaction Mode Score**: Captures the preferred ways a wallet interacts with the platform, such as through API, web, or mobile. This score is computed by analyzing the distribution of interaction types for each wallet.

- **Activity-Based Clustering (UMAP + HDBSCAN)**: Groups wallets into clusters based on their activity patterns using a two-step pipeline: UMAP for dimensionality reduction and HDBSCAN for density-based clustering. This model reveals natural groupings or archetypes in the user base, helping to identify segments with similar behaviors.